In [ ]:
import sys
sys.path.append("..")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import scipy.io.wavfile as wavfile
from scipy.signal import resample, correlate, hilbert
from PIL import Image


from util.plotting import compute_fft_plot_from_sample_rate
from util.data_io import read_rtl_raw_data, read_gqrx_raw_data
from util.filtering import low_pass_filter_complex_signal, low_pass_filter_real_signal
from util.demodulation import chunked_demodulate_signal
from util.phase_locked_loop import phase_locked_loop

In [ ]:
fs, signal = wavfile.read("/Users/benjaminpattison/Documents/SDRconnect_IQ_20240101_104718_137912500HZ.wav")
signal = signal[:,0] + 1j * signal[:,1]

fc = 137.1E6
len(signal) / fs

In [ ]:
start_index = 588994224 // 2
end_index = start_index + fs
cropped_signal = signal[start_index:end_index]

f,m = compute_fft_plot_from_sample_rate(cropped_signal, sampling_rate=fs)

In [ ]:
cropped_signal.shape, f.shape, m.shape

In [ ]:
fig = go.Figure()
fig.add_scattergl(x=f[::100],y=m[::100])
fig.show()

In [ ]:
audio, audio_fs = chunked_demodulate_signal(signal, fs, chunk_size=10, base_band_filter_cutoff=15E3,
                              base_band_downsample_rate=10, audio_filter_cutoff=15000,
                              audio_downsample_rate=5,
                              apply_output_filter=True)


# Alternative way of demodulating the FM signal using a PLL. This might be better able to compensate
# for doppler. However, it doesn't sound much better and it's way slower.

# filtered_signal = low_pass_filter_complex_signal(signal, sample_rate=fs, cutoff_frequency=40E3)
# _,_,frequencies = phase_locked_loop(filtered_signal, sampling_rate=fs, initial_frequency_estimate=0, frequency_bandwidth=40E3)

# filtered_frequencies = low_pass_filter_real_signal(frequencies, sample_rate=fs, cutoff_frequency=15E3)

# filtered_frequencies = filtered_frequencies[::50]
# audio_fs = fs / 50


In [ ]:
wavfile.write("/Users/benjaminpattison/Documents/Projects/satNav/SDR_satellite_tracking/data/2024_01_10_57_noaa18.wav", int(audio_fs), audio)

In [ ]:
# First, use a PLL to do synchronous AM demodulation.

# Crop to just the part that has a "good" sounding signal.
# 3:30-9:30
audio = audio[int(audio_fs * 210):int(audio_fs * 570)]

output, _, _ = phase_locked_loop(hilbert(audio), sampling_rate=audio_fs, initial_frequency_estimate=2.4E3, frequency_bandwidth=500)
am_demod_signal = np.real(audio * np.conjugate(output))

am_demod_signal = low_pass_filter_real_signal(am_demod_signal, sample_rate=audio_fs, cutoff_frequency=2080, order=50)

total_time = len(am_demod_signal) / audio_fs
total_pixels = np.floor(total_time * 4160).astype(int)
total_rows = total_pixels // 2080


resampled_signal = resample(am_demod_signal, total_pixels)
resampled_signal = resampled_signal[:total_rows*2080]

In [ ]:
"""
source: https://www.sigidwiki.com/wiki/Automatic_Picture_Transmission_(APT)
sync_a = 000011001100110011001100110011000000000
space_a = [0] * 47

sync_b = 000011100111001110011100111001110011100
space_b = [0] * 47
"""

# Stored as raw pixels at 4160 pixels / line.
sync_a_pixels = [-1, -1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
sync_b_pixels = [-1, -1, -1, -1, 1, 1, 1, -1, -1, 1, 1, 1, -1, -1, 1, 1, 1, -1, -1, 1, 1, 1, -1, -1, 1, 1, 1, -1, -1, 1, 1, 1, -1, -1, 1, 1, 1, -1, -1]

space_a = [-1] * 47
space_b = [1] * 47


sync_a_pixels = np.array(sync_a_pixels + space_a)
sync_b_pixels = np.array(sync_b_pixels + space_b)

In [ ]:
# Crop the signal to start at the first A frame.
start_time = 0
end_time = 3.0
start = int(4160 * start_time)
end = int(4160 * end_time)
crop = resampled_signal[start:end]
crop -= np.mean(crop)
crop /= np.max(np.abs(crop))


correlate_a = correlate(crop, sync_a_pixels)

start = np.argmax(correlate_a) - len(sync_a_pixels) + 1

num_rows = (len(resampled_signal) - start) // 2080
total_pixels = num_rows * 2080

cropped_signal = resampled_signal[start:start+total_pixels]


num_rows

In [ ]:
def get_row(signal, row_index):
    start_index = row_index * 2080
    end_index = start_index + 2080
    row_data = signal[start_index:end_index]

    low_value = np.percentile(row_data, 1)
    high_value = np.percentile(row_data, 99)

    normalized_row_data = (row_data - low_value) / (high_value - low_value)
    normalized_row_data = np.clip(normalized_row_data * 255, 0, 255).astype(np.uint8)
    return normalized_row_data.reshape([1, 2080])


rows = []

for i in range(num_rows):
    rows.append(get_row(cropped_signal, i))

image = np.concatenate(rows, axis=0)
plt.imsave("/Users/benjaminpattison/Documents/Projects/satNav/SDR_satellite_tracking/data/2024_01_10_57_noaa18.png", image, cmap='gray')


In [ ]:
plt.figure(figsize=(8,3))
plt.imshow(image, cmap='gray')

In [ ]:
goes = plt.imread("/Users/benjaminpattison/Downloads/20240011900_GOES18-ABI-FD-GEOCOLOR-10848x10848.jpg")
goes_cropped = goes[500:3000,5500:8000]
plt.imshow(goes_cropped)

In [ ]:
goes_cropped.shape

In [ ]:
resized_image = np.array(Image.fromarray(image).resize(size=(5000, 2500)))
resized_image = np.stack([resized_image]*3,axis=-1)

composite = np.zeros((5000,5000,3), dtype=np.uint8)
composite[:2500,:,:] = resized_image
composite[2500:,1250:3750,:] = goes_cropped


plt.imshow(composite)
plt.imsave("/Users/benjaminpattison/Documents/Projects/satNav/SDR_satellite_tracking/data/2024_01_10_57_noaa18_goes_composite.png", composite)


In [ ]:
# another reference signal
input_file_path = "data/NOAA1520190811-075637-40000_reference_fm_demod_alt.wav"
fs, signal = wavfile.read(input_file_path)

signal_cropped = signal[int(fs*1.5*60):int(fs*13.5*60)]

output, _, _ = phase_locked_loop(hilbert(signal_cropped), sampling_rate=fs, initial_frequency_estimate=2.4E3, frequency_bandwidth=500)
am_demod_signal = np.real(signal_cropped * np.conjugate(output))

am_demod_signal = low_pass_filter_real_signal(am_demod_signal, sample_rate=fs, cutoff_frequency=2080, order=50)

total_time = len(am_demod_signal) / fs
total_pixels = np.floor(total_time * 4160).astype(int)
total_rows = total_pixels // 2080


resampled_signal = resample(am_demod_signal, total_pixels)
resampled_signal = resampled_signal[:total_rows*2080]

In [ ]:
f,m = compute_fft_plot_from_sample_rate(output, sampling_rate=fs)
plt.plot(f,m)

In [ ]:
"""
source: https://www.sigidwiki.com/wiki/Automatic_Picture_Transmission_(APT)
sync_a = 000011001100110011001100110011000000000
space_a = [0] * 47

sync_b = 000011100111001110011100111001110011100
space_b = [0] * 47
"""

# Stored as raw pixels at 4160 pixels / line.
sync_a_pixels = [-1, -1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
sync_b_pixels = [-1, -1, -1, -1, 1, 1, 1, -1, -1, 1, 1, 1, -1, -1, 1, 1, 1, -1, -1, 1, 1, 1, -1, -1, 1, 1, 1, -1, -1, 1, 1, 1, -1, -1, 1, 1, 1, -1, -1]

space_a = [-1] * 47
space_b = [1] * 47


sync_a_pixels = np.array(sync_a_pixels + space_a)
sync_b_pixels = np.array(sync_b_pixels + space_b)

In [ ]:
len(sync_a_pixels), len(space_a)

In [ ]:
# Crop the signal to start at the first A frame.
start_time = 0
end_time = 3.0
start = int(4160 * start_time)
end = int(4160 * end_time)
crop = resampled_signal[start:end]
crop -= np.mean(crop)
crop /= np.max(np.abs(crop))


correlate_a = correlate(crop, sync_a_pixels)

start = np.argmax(correlate_a) - len(sync_a_pixels) + 1

num_rows = (len(resampled_signal) - start) // 2080
total_pixels = num_rows * 2080

cropped_signal = resampled_signal[start:start+total_pixels]


num_rows

In [ ]:
def get_row(signal, row_index):
    start_index = row_index * 2080
    end_index = start_index + 2080
    row_data = signal[start_index:end_index]

    low_value = np.percentile(row_data, 1)
    high_value = np.percentile(row_data, 99)

    normalized_row_data = (row_data - low_value) / (high_value - low_value)
    normalized_row_data = np.clip(normalized_row_data * 255, 0, 255).astype(np.uint8)
    return normalized_row_data.reshape([1, 2080])


rows = []

for i in range(num_rows):
    rows.append(get_row(cropped_signal, i))

image = np.concatenate(rows, axis=0)
plt.imsave("data/test_ref.png", image, cmap='gray')


In [ ]:
start = np.argmax(correlate_a) - len(sync_a_pixels) + 1
end = start + len(sync_a_pixels)
correlation_signal = np.zeros_like(correlate_a)
correlation_signal[start:end] = sync_a_pixels

In [ ]:
time_vector = np.linspace(start_time, end_time, len(crop))

fig = go.Figure()
fig.add_scatter(x=time_vector, y=crop, name="signal")
fig.add_scatter(x=time_vector, y=correlate_a/np.max(correlate_a), name="correlation")
fig.add_scatter(x=time_vector, y=correlation_signal, name="reference")


fig.show()

In [ ]:
import os
import imageio
import tqdm

In [ ]:
crop_time = 0.5
num_crops = int(len(signal_cropped) / fs / crop_time)
fig = plt.figure(figsize=(10,5))
output_dir = "data/figs/"

for i in tqdm.tqdm(range(num_crops)):
    start = int(i * fs * crop_time)
    end = int((i+1) * fs * crop_time)
    crop = signal_cropped[start:end]

    frequency_estimate = np.mean(frequencies[start:end]) * fs / (2*np.pi)

    f,m = compute_fft_plot_from_sample_rate(crop, sampling_rate=fs)

    plt.xlim((0, 5000))
    plt.ylim((0, 5000))
    plt.plot([frequency_estimate, frequency_estimate], [0, 5000])
    plt.plot(f,m)
    fig.savefig(os.path.join(output_dir, f"test{i}.png"))
    plt.clf()

plt.close()

In [ ]:
figs =[]
fig_file_names = os.listdir(output_dir)

for file_name in tqdm.tqdm(fig_file_names):
    image = plt.imread(os.path.join(output_dir, file_name))
    figs.append((image * 255).astype(np.uint8))

imageio.mimwrite(os.path.join(output_dir, "output.gif"), figs, fps=30)

In [ ]:
len(figs) / 30

In [ ]:
fig_file_names = os.listdir(output_dir)

for file_name in fig_file_names:
    figs.append(plt.imread(os.path.join(output_dir, file_name)))
    os.remove(os.path.join(output_dir, file_name))

In [ ]:
# MINE!
input_file_path = "data/gqrx_20220101_024640_137100000_2080000_fc.raw"
signal = read_gqrx_raw_data(input_file_path)
fs = 2080000
len(signal) / fs

audio, audio_fs = chunked_demodulate_signal(signal, fs, chunk_size=30, base_band_filter_cutoff=40E3,
                              base_band_downsample_rate=10, audio_filter_cutoff=15000,
                              audio_downsample_rate=5,
                              apply_output_filter=True)

wavfile.write("data/newest.wav", int(audio_fs), audio)

In [ ]:
audio_fs

In [ ]:
# another reference signal

signal_cropped = audio[int(audio_fs*1*60):int(audio_fs*8.5*60)]

output, _, _ = phase_locked_loop(hilbert(signal_cropped), sampling_rate=audio_fs, initial_frequency_estimate=2.4E3, frequency_bandwidth=500)
am_demod_signal = np.real(signal_cropped * np.conjugate(output))

am_demod_signal = low_pass_filter_real_signal(am_demod_signal, sample_rate=audio_fs, cutoff_frequency=2080, order=50)

total_time = len(am_demod_signal) / audio_fs
total_pixels = np.floor(total_time * 4160).astype(int)
total_rows = total_pixels // 2080


resampled_signal = resample(am_demod_signal, total_pixels)
resampled_signal = resampled_signal[:total_rows*2080]

In [ ]:
# Crop the signal to start at the first A frame.
start_time = 0
end_time = 3.0
start = int(4160 * start_time)
end = int(4160 * end_time)
crop = resampled_signal[start:end]
crop -= np.mean(crop)
crop /= np.max(np.abs(crop))


correlate_a = correlate(crop, sync_a_pixels)

start = np.argmax(correlate_a) - len(sync_a_pixels) + 1

num_rows = (len(resampled_signal) - start) // 2080
total_pixels = num_rows * 2080

cropped_signal = resampled_signal[start:start+total_pixels]


num_rows


def get_row(signal, row_index):
    start_index = row_index * 2080
    end_index = start_index + 2080
    row_data = signal[start_index:end_index]

    low_value = np.percentile(row_data, 1)
    high_value = np.percentile(row_data, 99)

    normalized_row_data = (row_data - low_value) / (high_value - low_value)
    normalized_row_data = np.clip(normalized_row_data * 255, 0, 255).astype(np.uint8)
    return normalized_row_data.reshape([1, 2080])


rows = []

for i in range(num_rows):
    rows.append(get_row(cropped_signal, i))

image = np.concatenate(rows, axis=0)
plt.imsave("data/mine_test.png", image, cmap='gray')

In [ ]:
973871319 / 2080000 / 2

In [ ]:
# With Ben's antenna!
input_file_path = "data/gqrx_20220109_025620_136741000_2080000_fc.raw"
signal = read_gqrx_raw_data(input_file_path)
fs = 2080000
len(signal) / fs

half_index = int(len(signal)/2)
signal = signal[:half_index]

In [ ]:
len(signal) / fs

In [ ]:
signal_length = len(signal)
start_index = 0
chunk_size = 30
chunk_sample_size = int(fs * chunk_size)
end_index = chunk_sample_size

shifted_signal = []


while end_index < signal_length:
    # Bring the end index back in case it has gone over.
    end_index = min(end_index, signal_length - 1)

    signal_crop = signal[start_index:end_index]

    offset_frequency = 363.1792E3
    time_vector = np.linspace(0, 30, len(signal_crop))
    offset_signal = np.exp(2j * np.pi * offset_frequency * time_vector)

    shifted_signal.append( signal_crop * offset_signal )




In [ ]:
shifted_signal = np.concatenate(shifted_signal)
len(shifted_signal)

In [ ]:
len(shifted_signal)

In [ ]:
cropped_signal = shifted_signal[:int(fs*1)]
f,m = compute_fft_plot_from_sample_rate(cropped_signal, sampling_rate=fs)

fig = go.Figure()
fig.add_scatter(x=f[::250],y=m[::250])
fig.show()

In [ ]:
offset_frequency = 363.1792E3
cropped_signal = signal[:int(fs*30)]


output, _, frequencies = phase_locked_loop(
    cropped_signal, sampling_rate=fs, initial_frequency_estimate=offset_frequency,
    frequency_bandwidth=40E3
)


frequencies_filtered = low_pass_filter_real_signal(frequencies, sample_rate=fs, cutoff_frequency=3000, order=50)
frequencies_filtered = frequencies_filtered[::10]
filtered_fs = fs / 10

In [ ]:
f,m = compute_fft_plot_from_sample_rate(frequencies_filtered, sampling_rate=filtered_fs)

fig = go.Figure()
fig.add_scatter(x=f[::250],y=m[::250])
fig.show()

In [ ]:
wavfile.write("data/newest.wav", int(filtered_fs), frequencies_filtered)

In [ ]:
audio, audio_fs = chunked_demodulate_signal(signal, fs, chunk_size=30, base_band_filter_cutoff=40E3,
                              base_band_downsample_rate=10, audio_filter_cutoff=15000,
                              audio_downsample_rate=5,
                              apply_output_filter=True)

wavfile.write("data/newest.wav", int(audio_fs), audio)